In [1]:
#!pip install torch torchvision

In [4]:
import torch
import torch.nn as nn

class S6Layer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(S6Layer, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, input_dim)
        self.activation = nn.GELU()

    def forward(self, x):
        # x: (batch_size, sequence_length, input_dim)
        h = self.activation(self.linear1(x))
        out = self.linear2(h)
        return out + x  # Residual connection

class VFIMamba(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(VFIMamba, self).__init__()
        self.s6_layers = nn.ModuleList([S6Layer(input_dim, hidden_dim) for _ in range(num_layers)])
        self.output_layer = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        for layer in self.s6_layers:
            x = layer(x)
        return self.output_layer(x)

import torch.optim as optim

# Hyperparameters
input_dim = 128 * 128 * 3  # Assuming 128x128 RGB images
hidden_dim = 512
num_layers = 4
learning_rate = 1e-4
num_epochs = 10

# Initialize model, loss function, and optimizer
model = VFIMamba(input_dim, hidden_dim, num_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for inputs, targets in dataloader:  # Define 'dataloader' to load your dataset
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


NameError: name 'dataloader' is not defined

In [ ]:
import cv2
import numpy as np

model.eval()
with torch.no_grad():
    for inputs, targets in test_dataloader:  # Define 'test_dataloader' for your test set
        outputs = model(inputs)
        # Reshape and convert outputs to images
        output_images = outputs.view(-1, 128, 128, 3).cpu().numpy()
        target_images = targets.view(-1, 128, 128, 3).cpu().numpy()
        # Compare output_images with target_images
        for i, (output_img, target_img) in enumerate(zip(output_images, target_images)):
            combined = np.hstack((output_img, target_img))
            cv2.imshow(f'Output vs Target {i}', combined)
            cv2.waitKey(0)
        cv2.destroyAllWindows()